In [30]:
import numpy as np
import pandas as pd

from sklearn.neighbors import KNeighborsRegressor

In [31]:
df = pd.read_csv("train.csv")
df = df.fillna(df.median(numeric_only=True))
print(df.shape)
df.head()

(73995, 72)


,user_id,org_id,rating,ts,user_city,org_city,average_bill,rating_org,rubrics,food_delivery,...,call_button,wheelchair_accessible,table_games,vinotheque,cakes_for_different_occasions,accepted_credit_cards,kalyan,teahouse,bread_from_tandoor,handmade_goods
0,12182789761588196310,11946401375069641937,4.0,677,msk,msk,1500.0,3.883212,Ресторан,1,...,0,0,0,0,0,0,0,0,0,0
1,2796544982129273287,5545859981678495568,5.0,577,msk,msk,1000.0,4.495379,Ресторан,1,...,0,0,0,0,0,0,0,0,0,0
2,10546824832088319728,3451485117888180206,4.0,716,msk,msk,500.0,4.576063,Кафе,1,...,0,0,0,0,0,0,0,0,0,0
3,9967742981458207920,9490041040735762889,3.0,639,spb,spb,1000.0,3.775701,Кафе,1,...,0,0,0,0,0,0,0,0,0,0
4,11046819495825038093,5234254278902729372,5.0,669,msk,msk,500.0,4.187500,Пиццерия,1,...,0,1,0,0,0,0,0,0,0,0


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73995 entries, 0 to 73994
Data columns (total 72 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   user_id                         73995 non-null  uint64 
 1   org_id                          73995 non-null  uint64 
 2   rating                          73995 non-null  float64
 3   ts                              73995 non-null  int64  
 4   user_city                       73995 non-null  object 
 5   org_city                        73995 non-null  object 
 6   average_bill                    73995 non-null  float64
 7   rating_org                      73995 non-null  float64
 8   rubrics                         73995 non-null  object 
 9   food_delivery                   73995 non-null  int64  
 10  breakfast                       73995 non-null  int64  
 11  takeaway                        73995 non-null  int64  
 12  summer_terrace                  

In [33]:
category_columns = ['rubrics', 'user_city', 'org_city']
train_category = pd.get_dummies(df[category_columns])
train_category

,rubrics_Бар безалкогольных напитков,"rubrics_Бар, паб","rubrics_Булочная, пекарня",rubrics_Быстрое питание,rubrics_Кальян-бар,rubrics_Кафе,rubrics_Кондитерская,rubrics_Кофе с собой,rubrics_Кофейня,rubrics_Пиццерия,rubrics_Ресторан,rubrics_Спортбар,rubrics_Столовая,rubrics_Суши-бар,user_city_msk,user_city_spb,org_city_msk,org_city_spb
0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0
1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0
2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0
3,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1
4,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73990,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
73991,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0
73992,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
73993,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0


In [34]:
df_corr = df.corr(numeric_only = True).abs()
df_corr

,user_id,org_id,rating,ts,average_bill,rating_org,food_delivery,breakfast,takeaway,summer_terrace,...,call_button,wheelchair_accessible,table_games,vinotheque,cakes_for_different_occasions,accepted_credit_cards,kalyan,teahouse,bread_from_tandoor,handmade_goods
user_id,1.000000,0.000468,0.009638,0.000955,0.002589,0.004119,0.000532,0.001360,0.001144,0.000847,...,0.000267,0.001250,0.004630,0.001086,0.001100,0.003231,0.006458,0.000313,0.000944,0.005174
org_id,0.000468,1.000000,0.002896,0.006500,0.006741,0.021809,0.002430,0.004482,0.008181,0.005189,...,0.005358,0.016988,0.008666,0.008409,0.016282,0.015557,0.015758,0.003586,0.002129,0.003089
rating,0.009638,0.002896,1.000000,0.008202,0.015849,0.269386,0.022417,0.000492,0.020302,0.004803,...,0.006382,0.005176,0.015679,0.000083,0.002001,0.005050,0.008119,0.008401,0.008872,0.011646
ts,0.000955,0.006500,0.008202,1.000000,0.010419,0.040647,0.000293,0.012898,0.084032,0.042755,...,0.008886,0.012276,0.009588,0.001499,0.018234,0.004507,0.005170,0.008811,0.026419,0.012300
average_bill,0.002589,0.006741,0.015849,0.010419,1.000000,0.044416,0.010670,0.022809,0.017473,0.054256,...,0.009249,0.009685,0.004273,0.004624,0.006851,0.003736,0.002746,0.004851,0.005118,0.005443
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
accepted_credit_cards,0.003231,0.015557,0.005050,0.004507,0.003736,0.015476,0.051693,0.034858,0.040969,0.036895,...,0.002882,0.002770,0.002102,0.001488,0.024186,1.000000,0.068134,0.022951,0.001747,0.083615
kalyan,0.006458,0.015758,0.008119,0.005170,0.002746,0.019148,0.033290,0.032068,0.024861,0.030379,...,0.012152,0.012854,0.022782,0.027268,0.001504,0.068134,1.000000,0.024195,0.001665,0.113265
teahouse,0.000313,0.003586,0.008401,0.008811,0.004851,0.022627,0.002606,0.001974,0.007209,0.006436,...,0.014145,0.059934,0.001145,0.001628,0.001728,0.022951,0.024195,1.000000,0.001912,0.031966
bread_from_tandoor,0.000944,0.002129,0.008872,0.026419,0.005118,0.014323,0.012294,0.022015,0.024931,0.021372,...,0.011844,0.006900,0.007309,0.002528,0.007446,0.001747,0.001665,0.001912,1.000000,0.001901


In [35]:
rating_corr: pd.Series = df_corr.get("rating_org").sort_values(ascending=False)
rating_corr = rating_corr.drop(labels=(['rating_org']))
rating_corr = rating_corr[rating_corr >= 0.001]
rating_corr

rating                    0.269386
cafe                      0.181062
toilet_for_disabled       0.173958
parking_disabled          0.167170
payment_by_credit_card    0.154419
                            ...   
vinotheque                0.007477
special_menu              0.006585
user_id                   0.004119
closed_for_quarantine     0.002378
breakfast                 0.001349
Name: rating_org, Length: 68, dtype: float64

In [36]:
from sklearn.model_selection import train_test_split

df1 = np.hstack([df[rating_corr.keys()], train_category])
X_train, X_test = train_test_split(df1, test_size=0.6)
Y_train, Y_test = train_test_split(df['rating_org'], test_size=0.6)

In [37]:
from sklearn.metrics import mean_absolute_percentage_error

p = 10
min_error = float('inf')

for n in (10, 1000, 100):
    reg = KNeighborsRegressor(n_neighbors=n, weights="distance")
    reg.fit(X_train, Y_train)

    pred = reg.predict(X_test)
    error = mean_absolute_percentage_error(Y_test, pred)

    if error < min_error:
        p = n
        min_error = error

print(min_error)

reg = KNeighborsRegressor(n_neighbors=p, weights="distance")
reg.fit(X_train, Y_train)

0.05514752740757852


KNeighborsRegressor(n_neighbors=1000, weights='distance')

In [38]:
df_test = pd.read_csv("test_x.csv")
df_test = df_test.fillna(df_test.median(numeric_only=True))
test_category = pd.get_dummies(df_test[category_columns])
X_test = np.hstack([df_test[rating_corr.keys()], test_category])

res = reg.predict(X_test)
res = pd.DataFrame(res, columns=['rating_org'])
res.index.name = 'id'
res.head()

,rating_org
id,
0,4.311033
1,4.337688
2,4.346611
3,4.320653
4,4.333461


In [39]:
res.to_csv("prediction.csv")